# conformer adapt

### Install dependencies

In [1]:
import os
import shutil

In [2]:
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install text-unidecode
!pip install matplotlib>=3.3.2
!pip install -q jiwer

## Install NeMo
BRANCH = 'main'
!python -m pip install -q git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

## Grab the config we'll use in this example
# !mkdir configs
# !wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/contextnet_rnnt/contextnet_rnnt.yaml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=0844a96c498fba21114aa00433406e4c350b2f994a4a64699bc4e10c6833f860
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-7ubuntu0.1).
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
The following additional packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox3
Suggested packages:
  libsox-fmt-all
The following NEW packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox3 sox
0 upgraded, 6 newly installed, 0 to remove and 24 not upgraded.


In [3]:
!pip install -q --upgrade gdown
!gdown 1EC679BsKJa2wL0uLMOWu_wecmrcJwneEGlmiySV9a4A
!gdown 124507E2eWFTXO3T9Au1CDQ0CjBv-vq8Y

shutil.unpack_archive('/content/raw-20230523T183259Z-001.zip')

Downloading...
From (uriginal): https://drive.google.com/uc?id=1EC679BsKJa2wL0uLMOWu_wecmrcJwneEGlmiySV9a4A
From (redirected): https://docs.google.com/spreadsheets/d/1EC679BsKJa2wL0uLMOWu_wecmrcJwneEGlmiySV9a4A/export?format=xlsx
To: /content/AI answers.xlsx
52.2kB [00:00, 82.7MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=124507E2eWFTXO3T9Au1CDQ0CjBv-vq8Y
From (redirected): https://drive.google.com/uc?id=124507E2eWFTXO3T9Au1CDQ0CjBv-vq8Y&confirm=t&uuid=20f571a1-e2af-4000-b0a2-cc46e91364a3
To: /content/raw-20230523T183259Z-001.zip
100% 215M/215M [00:05<00:00, 38.3MB/s]


### imports and preprocessing the data

In [1]:
import re
import os
import json
import shutil
import librosa
import numpy as np
import pandas as pd
import soundfile as sf

import torch
from jiwer import wer, cer
from omegaconf import OmegaConf, open_dict
from pytorch_lightning import Trainer

from nemo.utils import exp_manager
from nemo.core import adapter_mixins
import nemo.collections.asr as nemo_asr
from nemo.collections.common.parts.adapter_modules import LinearAdapterConfig

model_name = "stt_ru_conformer_ctc_large"
oov_regex = "[^ а-я]"
TRAIN_MANIFEST = "train_manifest.json"
TEST_MANIFEST = "test_manifest.json"

accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
device = 'cuda' if accelerator == 'gpu' else 'cpu'

[NeMo W 2023-05-24 11:27:16 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-05-24 11:27:18 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


some issue with original data

In [2]:
for file_name in os.listdir('raw'):
    aud, sr = librosa.load(f'raw/{file_name}', sr=16000)
    sf.write(f'raw/{file_name}', aud, 16000)

prepare dataset

In [3]:
raw_data = pd.read_excel('/content/AI answers.xlsx')
test_indexes = [4, 24, 29, 30]
raw_data_test = [raw_data['Transcriptions'][idx] for idx in test_indexes]

train_indexes = list(range(31))
[train_indexes.pop(idx) for idx in test_indexes[::-1]]
raw_data_train = [raw_data['Transcriptions'][idx] for idx in train_indexes]

# TEST
test_data = []
test_files = [f'raw/Audio{num}.wav' for num in np.array(test_indexes)+1]
aud_durations = [librosa.get_duration(y=librosa.load(path)[0], sr=16000) for path in test_files]

for i in range(len(test_files)):
    d = {}
    d['audio_filepath'] = test_files[i]
    d['duration'] = aud_durations[i]
    text = re.sub('\n', ' ', raw_data_test[i].lower())
    text = re.sub('-', ' ', text)
    text = re.sub(oov_regex, "", text).strip()
    text = re.sub(" +", " ", text)
    d['text'] = text
    test_data.append(d)

with open(TEST_MANIFEST, 'w') as f:
    for line in test_data:
        f.write(json.dumps(line) + '\n')

# TRAIN
train_data = []
train_files = [f'raw/Audio{num}.wav' for num in np.array(train_indexes)+1]
aud_durations = [librosa.get_duration(y=librosa.load(path)[0], sr=16000) for path in train_files]

for i in range(len(train_files)):
    d = {}
    d['audio_filepath'] = train_files[i]
    d['duration'] = aud_durations[i]
    text = re.sub('\n', ' ', raw_data_train[i].lower())
    text = re.sub('-', ' ', text)
    text = re.sub(oov_regex, "", text).strip()
    text = re.sub(" +", " ", text)
    d['text'] = text
    train_data.append(d)

with open(TRAIN_MANIFEST, 'w') as f:
    for line in train_data:
        f.write(json.dumps(line) + '\n')

### model setup

In [4]:
# Utility method to check and update the model config
def update_model_config_to_support_adapter(model_cfg):
    with open_dict(model_cfg):
        adapter_metadata = adapter_mixins.get_registered_adapter(model_cfg.encoder._target_)
        if adapter_metadata is not None:
            model_cfg.encoder._target_ = adapter_metadata.adapter_class_path
    
    print("Updated encoder _target_ model :", model_cfg.encoder._target_)
    return model_cfg

In [5]:
cfg = nemo_asr.models.ASRModel.from_pretrained(model_name, return_config=True)
cfg = update_model_config_to_support_adapter(cfg)

cfg.train_ds.batch_size=1
cfg.test_ds.batch_size=1

model = nemo_asr.models.ASRModel.from_pretrained(model_name, override_config_path=cfg)

[NeMo I 2023-05-24 11:27:21 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.19.0rc0/stt_ru_conformer_ctc_large/85481989be8cfb2f92a5bb9946b8da5c/stt_ru_conformer_ctc_large.nemo.
[NeMo I 2023-05-24 11:27:21 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.19.0rc0/stt_ru_conformer_ctc_large/85481989be8cfb2f92a5bb9946b8da5c/stt_ru_conformer_ctc_large.nemo
[NeMo I 2023-05-24 11:27:21 common:913] Instantiating model from pre-trained checkpoint
Updated encoder _target_ model : nemo.collections.asr.modules.conformer_encoder.ConformerEncoderAdapter
[NeMo I 2023-05-24 11:27:21 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.19.0rc0/stt_ru_conformer_ctc_large/85481989be8cfb2f92a5bb9946b8da5c/stt_ru_conformer_ctc_large.nemo.
[NeMo I 2023-05-24 11:27:21 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.19.0rc0/stt_ru_conformer_ctc_large/85481989be8cfb2f92a5bb9946b8da5c/stt_ru_conformer_ctc_large.nemo
[NeMo I 2023-05-24 11:27:21 common:913] Instan

[NeMo W 2023-05-24 11:27:23 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 1
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: ''
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    
[NeMo W 2023-05-24 11:27:23 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 16
    shuffle: false
    num_workers: 8
    pin_m

[NeMo I 2023-05-24 11:27:23 features:291] PADDING: 0
[NeMo I 2023-05-24 11:27:26 save_restore_connector:249] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.19.0rc0/stt_ru_conformer_ctc_large/85481989be8cfb2f92a5bb9946b8da5c/stt_ru_conformer_ctc_large.nemo.


### check metrics before adapting

In [ ]:
# temporarily save current model
# model.save_to("/content/unadapted_model.nemo")

In [ ]:
model.to(device)
model.eval()
with torch.no_grad():
    test_preds = model.transcribe([test_data[i]['audio_filepath'] for i in range(len(test_data))], batch_size=1)
    test_reals = [test_data[i]['text'] for i in range(len(test_data))]
    print('only test')
    print(np.round(wer(test_reals, test_preds), 4))
    print(np.round(cer(test_reals, test_preds), 4))

    train_preds = model.transcribe([train_data[i]['audio_filepath'] for i in range(len(train_data))], batch_size=1)
    train_reals = [train_data[i]['text'] for i in range(len(train_data))]
    print('all dataset')
    print(np.round(wer(test_reals+train_reals, test_preds+train_preds), 4))
    print(np.round(cer(test_reals+train_reals, test_preds+train_preds), 4))

Transcribing:   0%|          | 0/4 [00:00<?, ?it/s]

only test
0.1342
0.0484


Transcribing:   0%|          | 0/27 [00:00<?, ?it/s]

all dataset
0.1873
0.0934


only test
0.0846
0.0229
Transcribing: 100%
27/27 [00:11<00:00, 2.77it/s]
all dataset
0.0881
0.0313

In [ ]:
for i in range(len(test_data)):
    print('pred:', test_preds[i])
    print('real:', test_reals[i])
    print()

pred: добрый день техническая поддержка я с вгу слышу вас здравствуйте у меня не открывается блэкбод может же проверить подождите секунду сейчас проверим  да есть проблем с работой серверов работа будет восстанорена после проведения технических работ буду ждать до свидания до свидания
real: добрый день техническая поддержка спбгу слушаю вас здравствуйте у меня не открывается блэкборд можете проверить подождите секунду сейчас проверим да есть проблема с работой серверов работа будет восстановлена после проведения технических работ буду ждать до свидания до свидания

pred: здравствуйте это служба технической поддержки университета чем я могу вам помочь как очиститься жаль это слышать начисление это серьезное решение которое тбуют тщательного обдумывания вы все обдумали да я уверен в своем решениим в таком случае в личном кабинете в разделе заявки вам нужно создать новую заявку и вы проть тип заявки отчисление после заполнения ее нужно сохранить в вкладк документы хорошо я так и сделаю сп

### trainer setup

In [6]:
max_steps = 300

trainer = Trainer(devices=1, accelerator=accelerator, max_steps=max_steps,
                  enable_checkpointing=False, logger=False,
                  log_every_n_steps=50, check_val_every_n_epoch=1)
model.set_trainer(trainer)

model.cfg.train_ds.max_duration = 120.0

with open_dict(model.cfg):
    model.cfg.train_ds.manifest_filepath = TRAIN_MANIFEST
    model.cfg.train_ds.batch_size = 1
    model.cfg.train_ds.is_tarred = False
    model.cfg.train_ds.tarred_audio_filepaths = None

    model.cfg.validation_ds.manifest_filepath = TEST_MANIFEST
    model.cfg.validation_ds.batch_size = 1

model.setup_training_data(model.cfg.train_ds)
model.setup_multiple_validation_data(model.cfg.validation_ds)
model.setup_multiple_test_data(model.cfg.validation_ds)

with open_dict(model.cfg):
    model.cfg.optim.lr = 0.00002
    model.cfg.optim.weight_decay = 0.0
    model.cfg.optim.sched.warmup_steps = 150

model.setup_optimization(model.cfg.optim)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


[NeMo I 2023-05-24 11:27:27 collections:193] Dataset loaded with 24 files totalling 0.46 hours
[NeMo I 2023-05-24 11:27:27 collections:194] 3 files were filtered totalling 0.12 hours


[NeMo W 2023-05-24 11:27:27 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      warnings.warn(_create_warning_msg(
    


[NeMo I 2023-05-24 11:27:27 collections:193] Dataset loaded with 4 files totalling 0.07 hours
[NeMo I 2023-05-24 11:27:27 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-05-24 11:27:27 collections:193] Dataset loaded with 4 files totalling 0.07 hours
[NeMo I 2023-05-24 11:27:27 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-05-24 11:27:27 modelPT:721] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 2e-05
        maximize: False
        weight_decay: 0.0
    )
[NeMo I 2023-05-24 11:27:27 lr_scheduler:910] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7fc8502e4dc0>" 
    will be used during training (effective maximum steps = 300) - 
    Parameters : 
    (warmup_steps: 150
    warmup_ratio: null
    min_lr: 1.0e-06
    max_steps: 300
   

(AdamW (
 Parameter Group 0
     amsgrad: False
     betas: [0.9, 0.98]
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     initial_lr: 2e-05
     lr: 1.3245033112582784e-07
     maximize: False
     weight_decay: 0.0
 ),
 {'scheduler': <nemo.core.optim.lr_scheduler.CosineAnnealing at 0x7fc8502e4dc0>,
  'interval': 'step',
  'frequency': 1,
  'monitor': 'loss',
  'reduce_on_plateau': False})

### adapter setup

In [7]:
adapter_name = "tech_support"
adapter_dim = 386 
adapter_activation = "swish"
adapter_norm_position = "pre"

adapter_cfg = LinearAdapterConfig(
    in_features=model.cfg.encoder.d_model,  # conformer specific model dim. Every layer emits this dim at its output.
    dim=adapter_dim,  # the bottleneck dimension of the adapter
    activation=adapter_activation,  # activation used in bottleneck block
    norm_position=adapter_norm_position,  # whether to use LayerNorm at the beginning or the end of the adapter
)

model.add_adapter(name=adapter_name, cfg=adapter_cfg)

model.set_enabled_adapters(enabled=False)  # disable all adapters
model.set_enabled_adapters(name=adapter_name, enabled=True)  # enable only the current adapter we want to train

model.freeze()
model.unfreeze_enabled_adapters()

[NeMo I 2023-05-24 11:27:27 adapter_mixins:714] Setting adapter 'tech_support' status : Enabled = False
[NeMo I 2023-05-24 11:27:27 adapter_mixins:729] Setting adapter 'tech_support' status : Enabled = True
[NeMo I 2023-05-24 11:27:27 adapter_mixins:400] Froze module encoder.layers.0.conv.batch_norm: BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2023-05-24 11:27:27 adapter_mixins:400] Froze module encoder.layers.1.conv.batch_norm: BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2023-05-24 11:27:27 adapter_mixins:400] Froze module encoder.layers.2.conv.batch_norm: BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2023-05-24 11:27:27 adapter_mixins:400] Froze module encoder.layers.3.conv.batch_norm: BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2023-05-24 11:27:27 adapter_mixins:400] Froze module encoder.layers.4.conv.ba

In [8]:
os.environ.pop('NEMO_EXPM_VERSION', None)

exp_config = exp_manager.ExpManagerConfig(
    exp_dir=f'experiments/',
    name=f"ASR-Adapters",
    checkpoint_callback_params=exp_manager.CallbackParams(
        monitor="val_wer",
        mode="min",
        always_save_nemo=True,
        save_best_model=True,
    ),
)

exp_config = OmegaConf.structured(exp_config)

logdir = exp_manager.exp_manager(trainer, exp_config)

[NeMo I 2023-05-24 11:27:27 exp_manager:374] Experiments will be logged at experiments/ASR-Adapters/2023-05-24_11-27-27
[NeMo I 2023-05-24 11:27:27 exp_manager:797] TensorboardLogger has been set up


[NeMo W 2023-05-24 11:27:27 exp_manager:893] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to 300. Please ensure that max_steps will run for at least 1 epochs to ensure that checkpointing will not error out.


### fitness

In [ ]:
trainer.fit(model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2023-05-24 11:27:28 modelPT:721] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 2e-05
        maximize: False
        weight_decay: 0.0
    )
[NeMo I 2023-05-24 11:27:28 lr_scheduler:910] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7fc8500d2f50>" 
    will be used during training (effective maximum steps = 300) - 
    Parameters : 
    (warmup_steps: 150
    warmup_ratio: null
    min_lr: 1.0e-06
    max_steps: 300
    )


INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoderAdapter           | 128 M 
2 | decoder           | ConvASRDecoder                    | 66.2 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WERBPE                            | 0     
------------------------------------------------------------------------
7.1 M     Trainable params
121 M     Non-trainable params
128 M     Total params
514.538   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

[NeMo W 2023-05-24 11:27:36 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      warnings.warn(_create_warning_msg(
    


...

[NeMo I 2023-05-24 11:29:30 nemo_model_checkpoint:168] New best .nemo model saved to: /content/experiments/ASR-Adapters/2023-05-24_11-27-27/checkpoints/ASR-Adapters.nemo
[NeMo I 2023-05-24 11:29:39 nemo_model_checkpoint:168] New best .nemo model saved to: /content/experiments/ASR-Adapters/2023-05-24_11-27-27/checkpoints/ASR-Adapters.nemo


Validation: 0it [00:00, ?it/s]

[NeMo I 2023-05-24 11:29:59 wer_bpe:297] 
    
[NeMo I 2023-05-24 11:29:59 wer_bpe:298] reference:добрый день техническая поддержка спбгу слушаю вас здравствуйте у меня не открывается блэкборд можете проверить подождите секунду сейчас проверим да есть проблема с работой серверов работа будет восстановлена после проведения технических работ буду ждать до свидания до свидания
[NeMo I 2023-05-24 11:29:59 wer_bpe:299] predicted:добрый день техническая поддержка я с вгглу слышую вас здравствуйте у меня не открывается блэкборд может же проверить подождите секунду сейчас проверим  да есть проблем с работой серверов аа работа будет восставноверена после проведения технических работ буду ждать до свидания до свидания
[NeMo I 2023-05-24 11:30:00 wer_bpe:297] 
    
[NeMo I 2023-05-24 11:30:00 wer_bpe:298] reference:здравствуйте это служба технической поддержки университета чем я могу вам помочь как отчислиться жаль это слышать отчисление это серьезное решение которое требует тщательного обдумыван

INFO:pytorch_lightning.utilities.rank_zero:Epoch 3, global step 96: 'val_wer' was not in top 3


[NeMo I 2023-05-24 11:30:11 nemo_model_checkpoint:168] New best .nemo model saved to: /content/experiments/ASR-Adapters/2023-05-24_11-27-27/checkpoints/ASR-Adapters.nemo


### check metrics after adapting

In [ ]:
adapted_model_path = '/content/experiments/ASR-Adapters/2023-05-23_18-54-55/checkpoints/ASR-Adapters.nemo'
adapted_model = nemo_asr.models.ASRModel.restore_from(adapted_model_path)

In [ ]:
test_preds = adapted_model.transcribe([test_data[i]['audio_filepath'] for i in range(len(test_data))], batch_size=1)
test_reals = [test_data[i]['text'] for i in range(len(test_data))]
print('only test')
print(np.round(wer(test_reals, test_preds), 4))
print(np.round(cer(test_reals, test_preds), 4))

train_preds = adapted_model.transcribe([train_data[i]['audio_filepath'] for i in range(len(train_data))], batch_size=1)
train_reals = [train_data[i]['text'] for i in range(len(train_data))]
print('all dataset')
print(np.round(wer(test_reals+train_reals, test_preds+train_preds), 4))
print(np.round(cer(test_reals+train_reals, test_preds+train_preds), 4))

Transcribing:   0%|          | 0/4 [00:00<?, ?it/s]

only test
0.1342
0.0492


Transcribing:   0%|          | 0/27 [00:00<?, ?it/s]

all dataset
0.1871
0.093


In [ ]:
for i in range(len(train_data)):
    print('pred:', train_preds[i])
    print('real:', train_reals[i])
    print()

pred: добрый день техническая поддержка я сппгу слушаю вас добрый день я забыла к сожалению свой пароль почта что мне нужно сделать скажите пожалуйста свого фамилию имя отчество филатова мария евгеньевна хорошо какой у вас факультет мат мех какой у вас номер группы двадцать б ноль семь мм а хорошо завтрашнего дня можете зайти в деканат показать студенческий билет вам выдадут новый пароль а дистанционный никак не полочется это сделать простите но мы не можем без удостоверения личности выдать вам новый пароль ладно хорошо спасибо до свидания до свидания
real: добрый день техническая поддержка спбгу слушаю вас добрый день я забыла к сожалению свой пароль от почты что мне нужно сделать скажите пожалуйста свои фамилию имя отчество филатова мария евгеньевна хорошо какой у вас факультет матмех какой у вас номер группы двадцать б ноль семь мм ага хорошо с завтрашнего дня можете зайти в деканат показать студенческий билет и вам выдадут новый пароль а дистанционно никак не получится это сделать 